# 18. Getting started with roswtf
http://wiki.ros.org/ROS/Tutorials/Getting%20started%20with%20roswtf

## setup environment

In [1]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [3]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

In [4]:
# define package name and path
package_name = 'beginner_tutorials'

package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials'

## Checking your installation
roswtf examines your system to try and find problems. Let's try it out:

In [6]:
!roswtf

Loaded plugin tf.tfwtf
No package or stack in context
Static checks summary:

No errors or warnings

ROS Master does not appear to be running.
Online graph checks will not be run.
ROS_MASTER_URI is [http://localhost:11311]


## Trying it online
For this next step, we want a Master to be up, so go ahead and start a roscore.

## Running the roscore
Make sure that a roscore is up and running:
### roscore
Let's start by making sure that we have roscore running:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run roscore:
```
$ roscore
```

Now, try running the same sequence again:

In [8]:
!roswtf

Loaded plugin tf.tfwtf
No package or stack in context
Static checks summary:

No errors or warnings
Beginning tests of your ROS graph. These may take awhile...
analyzing graph...
... done analyzing graph
running graph rules...
... done running graph rules

Online checks summary:

Found 1 warning(s).
Warnings are things that may be just fine, but are sometimes at fault

WARNING The following node subscriptions are unconnected:
 * /rosout:
   * /rosout




roswtf did some online examination of your graph now that your roscore is running. Depending on how many ROS nodes you have running, this can take a long time to complete. As you can see, this time it produced a warning:
```
WARNING The following node subscriptions are unconnected:
 * /rosout:
   * /rosout
```   

roswtf is warning you that the rosout node is subscribed to a topic that no one is publishing to. In this case, this is expected because nothing else is running, so we can ignore it.

## Errors
roswtf will warn you about things that look suspicious but may be normal in your system. It can also report errors for problems that it knows are wrong.

For this part, we are going to set your ROS_PACKAGE_PATH to a bad value. We're also going to stop our roscore to simplify the output that you see.
```
$ roscd
$ ROS_PACKAGE_PATH=bad:$ROS_PACKAGE_PATH roswtf
```

This time we see:
```
Stack: ros
======================================================
Static checks summary:

Found 1 error(s).

ERROR Not all paths in ROS_PACKAGE_PATH [bad] point to an existing directory: 
 * bad

======================================================
```

Cannot communicate with master, ignoring graph checks
As you can see, roswtf now gives us an error about the ROS_PACKAGE_PATH setting.

There are many other types of problems that roswtf can find. If you find yourself stumped by a build or communication issue, try running it and seeing if it can point you in the right direction.

Now that you know how to use roswtf, take sometime to learn more about how ros.org is structured and navigating the wiki.